# HW5: The Marvel Universe!

In [104]:
import networkx as nx
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

### Load the Data

In [2]:
data_edges=pd.read_csv('edges.csv')

In [3]:
data_nodes=pd.read_csv('nodes.csv')

In [4]:
data_heroes=pd.read_csv('hero-network.csv')

In [5]:
data_edges.head()

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251


In [6]:
data_nodes.head()

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero


In [7]:
data_heroes.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


In [8]:
len(data_nodes)

19090

## Preprocessing the Data

In [9]:
data_heroes

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
...,...,...
574462,COLOSSUS II/PETER RA,CALLISTO
574463,CALLISTO,ROGUE /
574464,CALLISTO,CALIBAN/
574465,CALIBAN/,ROGUE /


In [10]:
new_heroes1=data_heroes.hero1

In [11]:
aux='HULK/DR. ROBERT BRUC             '

In [12]:
' '.join(re.split(r'\s+', aux)).rstrip(' ')

'HULK/DR. ROBERT BRUC'

To remove the extra blank spaces at the end of the heroes' names

In [13]:
data_heroes.hero1=[' '.join(re.split(r'\s+', data_heroes.hero1[i])).rstrip(' ') for i in range(len(data_heroes))]

In [14]:
data_heroes.hero2=[' '.join(re.split(r'\s+', data_heroes.hero2[i])).rstrip(' ') for i in range(len(data_heroes))]

In [15]:
data_heroes

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
...,...,...
574462,COLOSSUS II/PETER RA,CALLISTO
574463,CALLISTO,ROGUE /
574464,CALLISTO,CALIBAN/
574465,CALIBAN/,ROGUE /


Here we remove the `/` at the end of the heroes' names

In [16]:
txt='CALIBAN/'

In [17]:
txt.rstrip('/')

'CALIBAN'

In [18]:
data_heroes.hero1=[data_heroes.hero1[i].rstrip('/') for i in range(len(data_heroes))]
data_heroes.hero2=[data_heroes.hero2[i].rstrip('/') for i in range(len(data_heroes))]

In [19]:
data_heroes

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
...,...,...
574462,COLOSSUS II/PETER RA,CALLISTO
574463,CALLISTO,ROGUE
574464,CALLISTO,CALIBAN
574465,CALIBAN,ROGUE


Here we check the rows that would have been a loop in the graph (hero1=hero2) and drop them

In [72]:
aux=[]
for i in range(len(data_heroes)):
    if data_heroes.hero1[i]==data_heroes.hero2[i]:
        aux.append(i)

In [78]:
data_heroes.drop(aux,inplace=True)

Now we make the corresponding modifications for the superhero `SPIDER-MAN`:

In [80]:
data_heroes.reset_index()

,index,hero1,hero2
0,0,"LITTLE, ABNER",PRINCESS ZANDA
1,1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,3,"LITTLE, ABNER",PRINCESS ZANDA
4,4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
...,...,...,...
572230,574462,COLOSSUS II/PETER RA,CALLISTO
572231,574463,CALLISTO,ROGUE
572232,574464,CALLISTO,CALIBAN
572233,574465,CALIBAN,ROGUE


In [82]:
data_heroes=data_heroes[['hero1','hero2']]

In [94]:
for i in tqdm(range(len(data_heroes))):
    if data_heroes.hero1.iloc[i]=='SPIDER-MAN/PETER PAR':
        data_heroes.hero1.iloc[i]='SPIDER-MAN/PETER PARKER'
    if data_heroes.hero2.iloc[i]=='SPIDER-MAN/PETER PAR':
        data_heroes.hero2.iloc[i]='SPIDER-MAN/PETER PARKER'
        

100%|████████████████████████████████| 572235/572235 [00:33<00:00, 17220.89it/s]


In [111]:
data_heroes.head()

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL


Since some rows have been removed, from now on we shall work with `iloc`